In [ ]:
import numpy as np
import plotly.graph_objects as go

# Parameters for the spinning black hole
mass = 1.0
spin = 3.2
grid_size = 20
speed_of_light = 299792458
observer_velocity = np.array([0.5, 0.0, 0.0])

x = np.linspace(-2, 2, grid_size)
y = np.linspace(-2, 2, grid_size)
z = np.linspace(-2, 2, grid_size)
X, Y, Z = np.meshgrid(x, y, z)

fig = go.Figure()

num_frames = 70
t_values = np.linspace(0, 2 * np.pi, num_frames)
delay_factor = 0.01

# Calculate the gravitational wave vectors at each point in the grid
def calculate_gravitational_wave_vector(x, y, z, t):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(y, x)
    phi = np.arctan2(z, np.sqrt(x**2 + y**2))
    
    amplitude = np.sin(theta + spin * r - t)
    direction_x = -np.sin(phi) * np.cos(theta) * amplitude
    direction_y = -np.sin(phi) * np.sin(theta) * amplitude
    direction_z = np.cos(phi) * amplitude
    
    return direction_x, direction_y, direction_z

# Create a 3D scatter plot for the gravitational wave vectors
fig = go.Figure(data=go.Cone(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    u=np.zeros_like(X.flatten()),
    v=np.zeros_like(Y.flatten()),
    w=np.zeros_like(Z.flatten()),
    sizemode="absolute",
    sizeref=0.1,  # Adjust cone size for visibility
    anchor="tail",
    colorscale='Viridis',  # Use a uniform color scale
    showscale=False,  # Hide the color scale
))

# Add a marker for the black body 
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers',
                           marker=dict(color='black', size=5, opacity=1)))

# Add the axis of rotation for the spinning black body
axis_length = 2.5
fig.add_trace(go.Scatter3d(x=[0, 0], y=[0, 0], z=[-axis_length, axis_length], mode='lines',
                           line=dict(color='red', width=1)))

# Set plot title and axis labels
fig.update_layout(title='Gravitational Wave Vectors and Axis of Rotation around a Spinning Black Hole',
                  scene=dict(
                      xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
                      aspectratio=dict(x=1, y=1, z=1),
                  ))

frames = []

for i, t in enumerate(t_values):
    U, V, W = calculate_gravitational_wave_vector(X, Y, Z, t - (delay_factor * i / num_frames) * 2 * np.pi * mass / speed_of_light)
    
    # Calculate opacity based on propagation delay
    opacity = (i / num_frames) * 0.8 + 0.2
    
    frame_data = go.Cone(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        u=U.flatten(),
        v=V.flatten(),
        w=W.flatten(),
        sizemode="absolute",
        sizeref=0.82,  # Adjust cone size for visibility
        anchor="tail",
        colorscale='edge',  # Use a uniform color scale
        showscale=True,  # Hide the color scale
        opacity=opacity,  # Apply opacity based on propagation delay
    )
    frame = go.Frame(data=[frame_data])
    frames.append(frame)

fig.frames = frames

fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(
                    label='Play',
                    method='animate',
                    args=[
                        None,
                        {
                            'frame': {'duration': 50, 'redraw': True},
                            'fromcurrent': True,
                            'transition': {'duration': 0}
                        }
                    ]
                ),
                dict(
                    label='Pause',
                    method='animate',
                    args=[
                        [None],
                        {
                            'frame': {'duration': 0, 'redraw': False},
                            'mode': 'immediate',
                            'transition': {'duration': 0}
                        }
                    ]
                )
            ],
            showactive=False,
            x=0.1,
            y=0,
            xanchor='right',
            yanchor='top'
        )
    ]
)

# Display the plot
fig.show()
